# **Transformer 기반 한국어 뉴스 요약 모델 학습 및 평가**

## 1. 프로젝트 개요

본 프로젝트는 Transformer 기반 Seq2Seq 모델을 활용하여  
한국어 뉴스 기사에 대한 자동 요약 모델을 구현하는 것을 목표로 한다.  

Pretrained 모델을 기반으로 Fine-tuning을 수행하였으며,  
학습 이후 Validation 데이터셋을 통해 모델 성능을 평가하였다.

---

## 2. 프로젝트 목표

- 한국어 뉴스 요약을 위한 Abstractive Summarization 모델 구현
- Pretrained Transformer 모델 Fine-tuning
- Validation Set 기준 ROUGE 지표를 통한 성능 평가
- 학습 및 추론 파이프라인 구현

---

## 3. 데이터 구성

- 입력(Input): 뉴스 기사 본문
- 출력(Target): 해당 기사에 대한 요약 문장
- Train / Validation 데이터 분리 후 학습 진행
- 입력 길이 제한 및 토큰 truncation 적용

---

## 4. 모델 학습

- Transformer 기반 Encoder–Decoder 구조 사용
- Fine-tuning을 통해 뉴스 도메인에 맞게 모델 적응
- Beam Search 기반 텍스트 생성

---

## 5. 성능 평가

학습 완료 후 Validation 데이터에 대해 요약을 생성하고  
ROUGE-1, ROUGE-2, ROUGE-L 지표를 통해 모델 성능을 평가하였다.  

이를 통해 모델의 핵심 단어 재현 능력과 문장 수준의 유사도를 확인하였다.

---

※ 본 파일은 모델 학습(Training)과 Validation 평가(Evaluation)까지의 과정을 포함한다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/iNES_project
!ls

/content/drive/MyDrive/iNES_project
final_model    results		     train.py
preprocess.py  train_processed.json  valid_processed.json


In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
!pip install transformers datasets torch tqdm

# **텍스트 요약 모델 학습 스크립트 (KoBART)**

**0. 환경 및 라이브러리 설정**
* 코드를 실행하면 `train.py` 파이썬 파일로 저장
* 데이터 처리용 `datasets` 및 모델 학습용 `transformers` 라이브러리를 로드

**1. 데이터 로드 및 샘플링**
* 전처리된 JSON 데이터를 로드
* 빠른 테스트 학습을 위해 **상위 5,000개 데이터만 샘플링**하여 `Dataset` 객체로 변환

**2. 모델 및 토크나이저 로드**
* 한국어 요약에 특화된 **KoBART 모델**`digit82/kobart-summarization`을 베이스 모델로 사용
* 텍스트 처리를 위한 토크나이저와 Seq2Seq 생성 모델을 불러옴

**3. 데이터 전처리 (토크나이징)**
* **원문은 최대 512 토큰, 요약문은 최대 128 토큰**으로 길이를 맞춤 (패딩 및 자르기 적용)
* 모델 학습 정답지로 쓰기 위해 요약문을 `labels`로 입력 데이터에 추가

**4. 학습 환경 설정 (Training Arguments)**
* **배치 사이즈 4, 에폭(Epoch) 1**로 빠른 테스트용 학습을 세팅
* **`fp16=True`**를 적용해 GPU 메모리를 아끼고 학습 속도를 크게 높임

**5 & 6. 학습 진행 및 모델 저장**
* `Trainer`를 이용해 **모델 학습(파인튜닝)을 진행**
* 학습이 끝난 최종 모델과 토크나이저를 추론용으로 `./final_model` 폴더에 저장

In [5]:
%%writefile train.py

import json
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Trainer,
    TrainingArguments
)

# 1. 데이터 로드
with open("train_processed.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 먼저 5000개로 자르기
data = data[:5000]

dataset = Dataset.from_list(data)

# 2. 모델 선택
model_name = "digit82/kobart-summarization"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 3. 토크나이징
def preprocess(example):
    # 원문 토큰화
    inputs = tokenizer(
        example["input_text"],
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # 타겟 요약 토큰화
    labels = tokenizer(
        example["target_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=False)

# 4. 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

# 5. 학습 시작
trainer.train()

# 6. 모델 저장
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")

Overwriting train.py


In [6]:
!python train.py

config.json: 1.20kB [00:00, 757kB/s]
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
tokenizer_config.json: 100% 295/295 [00:00<00:00, 1.59MB/s]
tokenizer.json: 682kB [00:00, 23.1MB/s]
special_tokens_map.json: 100% 109/109 [00:00<00:00, 615kB/s]
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
pytorch_model.bin: 100% 496M/496M [00:05<00:00, 96.6MB/s]
model.safetensors:   0% 0.00/496M [00:00<?, ?B/s]
Loading weights:   0% 0/262 [00:00<?, ?it/s]
Loading weights:   0% 1/262 [00:00<00:00, 11244.78it/s, Materializing param=final_logits_bias]
Loading weights:   0% 1/262 [00:00<00:00, 5398.07it/s, Materializing param=final_logits_bias] 
Loading weights:   1% 2/262 [00:00<00:00, 5614.86it/s, Materializing param=model.decoder.embed_positions.weight]
Loading weights:   1% 2/262 [00:00<00:00, 4514.86it/s, Materializing param=model.decoder.embed_positions.weight]
Loading weights:   1% 3/262 [00:00<00:00, 4753.65it/s, Material

# **학습 완료 모델 로드 및 요약 테스트 (추론)**

**1. 파인튜닝된 모델 불러오기**
* 학습 후 `./final_model` 폴더에 저장해둔 **최종 모델과 토크나이저를 로드**

**2. 검증 데이터(Validation) 준비**
* 테스트를 위해 `valid_processed.json` 파일을 로드
* 데이터 중 **첫 번째 문서의 원문**(`input_text`)을 가져와 테스트 문장으로 사용

**3. 텍스트 추론 및 요약 생성**
* 원문을 모델이 이해할 수 있도록 토크나이징 (최대 512 토큰 제한)
* **`model.generate()`** 함수로 요약 텍스트를 생성 (`num_beams=4`를 적용해 더 자연스럽고 퀄리티 높은 문장을 탐색, 최대 128 토큰)
* 생성된 결과값(ID)을 사람이 읽을 수 있는 일반 텍스트로 디코딩 (특수 토큰 제거)

**4. 결과 확인**
* 입력한 원문의 앞부분과 모델이 **직접 생성한 요약문을 화면에 출력하여 성능을 확인**

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 학습 끝난 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained("./final_model")
model = AutoModelForSeq2SeqLM.from_pretrained("./final_model")


# 예시: validation 데이터에서 첫 문서 가져오기
import json

with open("valid_processed.json", "r", encoding="utf-8") as f:
    valid_data = json.load(f)

text = valid_data[0]["input_text"]  # 첫 번째 문서 원문
inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(**inputs, max_length=128, num_beams=4)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("원문:", text[:200], "...")  # 길면 앞부분만
print("생성 요약:", summary)

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Loading weights:   0%|          | 0/262 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


원문: [ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 "넓고, 체증 없는 '통신 고속도로'가 5G"라며 "대한민국의 대전환이 이제 막 시작됐다"고 기대감을 높였다. 문 대통령은 8일 서울 올림픽공원에서 열린 5G플러스 전략발표에 참석해 "5G 시대는 우리가 생각하고, 만들면 그것이 세계 표준이 되는 시대"라며 "5G는 대한민국 혁신성장의  ...
생성 요약: '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 5G가 4차 산업혁명 시대의 고속도로가 돼 새로운 기회를 열어 줄 것이라고 강조했다.


# **검증 데이터 전체 요약 및 ROUGE 평가 (CSV 저장)**

**1. 모델 로드 및 평가 환경 세팅**
* 학습이 끝난 최종 모델(`./final_model`)을 불러와 **GPU(`cuda`)에 할당**
* 모델을 평가 모드(`model.eval()`)로 전환하고 검증용 데이터(`valid_processed.json`)를 로드

**2. 배치 단위 요약 생성 (속도 최적화)**
* 추론 속도를 높이기 위해 **배치 사이즈 16, 빔 서치(`num_beams`) 1, 최대 입력 길이 384**로 세팅
* `tqdm`으로 진행률을 확인하며, 메모리 절약을 위해 **`torch.no_grad()`** 상태에서 배치 단위로 요약문을 생성

**3. 모델 성능 평가 (ROUGE Score)**
* Hugging Face의 **`evaluate` 라이브러리를 사용해 ROUGE 점수를 계산**
* 실제 정답 요약문(Reference)과 모델이 생성한 요약문(Prediction)을 비교해 객관적인 요약 성능을 수치화

**4. 평가 결과 CSV 파일 저장**
* 원문, 정답 요약문, 생성 요약문을 묶어 **Pandas DataFrame**으로 변환
* 엑셀에서 한글 깨짐을 방지하기 위해 `encoding="utf-8-sig"`를 적용하여, **`validation_summaries.csv` 파일로 저장**

In [8]:
# 전체 validation 요약 + ROUGE 계산 + CSV 저장

!pip install evaluate
!pip install rouge_score

import json
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import evaluate

# 1. 모델 로드
model_path = "./final_model"  # 학습한 모델 경로
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# 2. validation 데이터 로드
with open("valid_processed.json", "r", encoding="utf-8") as f:
    valid_data = json.load(f)

# 3. 배치 요약 생성 (속도 최적화)
batch_size = 16       # GPU VRAM 여유 있으면 32 가능
num_beams = 1         # 속도 위해 빔서치 1
max_input_len = 384   # 입력 토큰 길이 줄임
max_output_len = 128

generated_summaries = []

for i in tqdm(range(0, len(valid_data), batch_size), desc="Generating summaries"):
    batch = valid_data[i:i+batch_size]
    texts = [doc["input_text"] for doc in batch]

    # 배치 토크나이징
    inputs = tokenizer(
        texts,
        return_tensors="pt",
        max_length=max_input_len,
        truncation=True,
        padding=True
    ).to(device)

    with torch.no_grad():
        summary_ids = model.generate(
            **inputs,
            max_length=max_output_len,
            num_beams=num_beams,
            early_stopping=True
        )

    batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    generated_summaries.extend(batch_summaries)

print("배치 요약 완료, 총 생성 개수:", len(generated_summaries))

# 4. ROUGE 계산
rouge = evaluate.load("rouge")  # ← datasets 대신 evaluate 사용

references = [doc["target_text"] for doc in valid_data]
results = rouge.compute(predictions=generated_summaries, references=references)
print("🔹 ROUGE 결과:", results)

# 5. 결과 CSV 저장
df = pd.DataFrame({
    "input_text": [doc["input_text"] for doc in valid_data],
    "target_text": references,
    "generated_summary": generated_summaries
})

df.to_csv("validation_summaries.csv", index=False, encoding="utf-8-sig")
print("결과 CSV 저장 완료: validation_summaries.csv")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 721.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=7ea97f271bf10e20cd1421041b98925ec1774c01d4b99e230f28fb8632437256
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


Loading weights:   0%|          | 0/262 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.shared.weight to model.decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie model.shared.weight to model.encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
Generating summaries: 100%|██████████| 1883/1883 [45:46<00:00,  1.46s/it]


✅ 배치 요약 완료! 총 생성 개수: 30122


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


🔹 ROUGE 결과: {'rouge1': np.float64(0.43219674597645785), 'rouge2': np.float64(0.17804824548746), 'rougeL': np.float64(0.41315382344596097), 'rougeLsum': np.float64(0.4133655222460041)}
✅ 결과 CSV 저장 완료: validation_summaries.csv
